In [3]:
!pip install arxiv


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: C:\Users\hwnam\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [18]:
import urllib
import arxiv
import requests
import json
import csv
import pandas as pd
from collections import Counter, defaultdict
import numpy as np # for array manipulation
import matplotlib.pyplot as plt # for data visualization
%matplotlib inline 
import datetime
import json
import tarfile
import os
import re

In [6]:
# read filtered_papers.json
with open('filtered_papers.json', 'r', encoding="utf-8") as f:
    papers = json.load(f)

print(f"Number of papers: {len(papers)}")

Number of papers: 2252


In [9]:
paper = papers[0]
query = paper["title"]
print(f"Query: {query}")

search = arxiv.Search(
    query=query,
    max_results=1,
    sort_by=arxiv.SortCriterion.Relevance,
)

for result in search.results():
    title = result.title
    paper_id = result.entry_id.split("/")[-1]
    print(f"Found paper: {title}")
    print(f"Paper_id: {paper_id}")

    if query == title:
        max_try = 0
        print("Paper found in arXiv")
        
        url = f"https://arxiv.org/e-print/{paper_id}"
        response = requests.get(url)
        while max_try < 5:
            if response.status_code == 200:
                with open(f"{paper_id}.tar.gz", "wb") as f:
                    f.write(response.content)
                print("Paper found source file")
                break
            else:
                print("Paper not found source file")
                max_try += 1
                response = requests.get(url)

Query: Residual Attention Network for Image Classification


C:\Users\hwnam\AppData\Local\Temp\ipykernel_42256\3935089147.py:11: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Found paper: Residual Attention Network for Image Classification
Paper_id: 1704.06904v1
Paper found in arXiv
Paper found source file


In [14]:
tar_path = "1704.06904v1.tar.gz"

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall("Example")


for fname in os.listdir("Example"):
    if fname.endswith(".tex"):
        print(fname)
        tex_file = fname

attention-net_camera_ready_merge_final.tex


C:\Users\hwnam\AppData\Local\Temp\ipykernel_42256\2916178336.py:4: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("Example")


In [28]:
def split_section(tex_file):
    pattern = r"\\section\*?{([^}]+)}"
    matches = list(re.finditer(pattern, tex_file))
    print(f"Number of sections: {len(matches)}")
    
    sections = []
    for i, match in enumerate(matches):
        section_name = match.group(1).strip()
        
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(tex_file)
        
        content = tex_file[start:end].strip()
        sections.append((section_name, content))
        
    return sections

In [29]:
with open("Example/" + tex_file, "r", encoding="utf-8") as f:
    latex_text = f.read()

splitBy_section = split_section(latex_text)

splitBy_section

Number of sections: 5


[('Introduction',
  '\\begin{figure*}\n\\begin{center}\n%\\fbox{\\rule{0pt}{2in} \\rule{.9\\linewidth}{0pt}}\n\\includegraphics[width=1\\linewidth]{motivation.pdf}\n\\end{center}\n   \\caption{\\textbf{Left:} an example shows the interaction between features and attention masks. \\textbf{Right:} example images illustrating that different features have different corresponding attention masks in our network. The sky mask diminishes low-level background blue color features. The balloon instance mask highlights high-level balloon bottom part features.}\n\\label{fig:motivation}\n\\end{figure*}\n\nNot only a friendly face but also red color will draw our attention. The mixed nature of attention has been studied extensively in the previous literatures~\\cite{walther2002attentional, itti2001computational,mnih2014recurrent,zhao2016diversified}. Attention not only serves to select a focused location but also enhances different representations of objects at that location. Previous works formulate

In [41]:
json_data = []
for section_name, content in splitBy_section:
    cleaned_content = re.sub(r"\\begin{([a-zA-Z*]+)}(?:\[[^\]]*\])?.*?\\end{\1}"
, "", content, flags=re.DOTALL)
    cleaned_content = cleaned_content.strip()
    
    lines = cleaned_content.splitlines()
    cleaned_content = [line for line in lines if line.strip() and not line.startswith("%")]
    cleaned_content = "\n".join(cleaned_content)
    
    json_data.append({
        "section_name": section_name,
        "content": cleaned_content
    })
    
with open("Example/cleaned_data.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [43]:
print(cleaned_content)

We propose a Residual Attention Network which stacks multiple Attention Modules. The benefits of our network are in two folds: it can capture mixed attention and is an extensible convolutional neural network. The first benefit lies in that different Attention Modules capture different types of attention to guide feature learning. Our experiments on the forms of activation function also validate this point: free form mixed attention will have better performance than constrained (including single) attention. The second benefit comes from encoding top-down attention mechanism into bottom-up top-down feedforward convolutional structure in each Attention Module. Thus, the basic Attention Modules can be combined to form larger network structure. Moreover, residual attention learning allows training very deep Residual Attention Network. The performance of our model surpasses state-of-the-art image classification methods, \ie ResNet on CIFAR-10 (3.90\% error), CIFAR-100 (20.67\% error), and ch